<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/08_Busqueda_Dispersa_y_Densa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Búsqueda Dispersa y Densa de Información sobre Películas en Español

Este notebook implementa un recuperador de un sistema RAG que combina:
1. Búsqueda dispersa (sparse retrieval) usando BM25
2. Búsqueda densa (dense retrieval) usando embeddings
3. Fusión de resultados

La búsqueda dispersa es buena para encontrar coincidencias exactas y palabras clave,
mientras que la búsqueda densa es mejor para capturar similitud semántica.

## Importamos las bibliotecas necesarias

In [1]:
!pip install rank_bm25
import pandas as pd
import numpy as np
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import stopwords
import re
from typing import List, Dict, Tuple

In [2]:
print("Descargando recursos de NLTK...")
import nltk

# Descarga de recursos necesarios
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)
print("Recursos NLTK descargados correctamente.")


Descargando recursos de NLTK...
Recursos NLTK descargados correctamente.


## 1. Preparación de Datos

Creamos un conjunto de datos de ejemplo con películas en español.

In [3]:
peliculas_data = {
    'titulo': [
        'El Padrino',
        'Pulp Fiction',
        'El Señor de los Anillos',
        'Matrix',
        'Ciudad de Dios',
        'El Laberinto del Fauno',
        'Amores Perros',
        'Todo Sobre Mi Madre',
        'Cinema Paradiso',
        'El Secreto de Sus Ojos'
    ],
    'descripcion': [
        'La película sigue a la familia Corleone, una de las familias más poderosas de la mafia italiana en Nueva York.',
        'Varias historias entrelazadas sobre criminales de Los Ángeles, incluyendo dos hitmen, un boxeador y un gángster.',
        'Un hobbit debe destruir un anillo mágico para salvar a la Tierra Media de la oscuridad.',
        'Un programador descubre que el mundo en el que vive es una simulación controlada por máquinas.',
        'La historia del crecimiento del crimen organizado en los suburbios de Río de Janeiro.',
        'En la España de 1944, una niña escapa a un mundo mágico y misterioso para huir de su padrastro y la guerra civil.',
        'Tres historias diferentes en Ciudad de México se entrelazan en torno a un accidente de coche.',
        'Una madre busca al padre del hijo que acaba de perder para comunicarle la muerte de su hijo.',
        'Un famoso director de cine recuerda su infancia y su primer amor por el cine en un pequeño pueblo italiano.',
        'Un oficial judicial se obsesiona con un caso de asesinato sin resolver durante 25 años.'
    ],
    'genero': [
        'Drama, Crimen',
        'Crimen, Drama',
        'Fantasía, Aventura',
        'Ciencia Ficción, Acción',
        'Crimen, Drama',
        'Fantasía, Drama',
        'Drama, Thriller',
        'Drama',
        'Drama, Romance',
        'Drama, Misterio'
    ],
    'año': [1972, 1994, 2001, 1999, 2002, 2006, 2000, 1999, 1988, 2009]
}

df = pd.DataFrame(peliculas_data)

## 2. Preprocesamiento de Texto

In [4]:
class TextPreprocessor:
    """
    Clase para preprocesar texto en español.
    """
    def __init__(self):
        self.stop_words = set(stopwords.words('spanish'))

    def preprocess(self, text: str) -> str:
        """Preprocesa un texto en español."""
        # Convertir a minúsculas
        text = text.lower()

        # Tokenizar usando NLTK
        tokens = word_tokenize(text)

        # Eliminar stopwords usando NLTK
        stop_words = set(stopwords.words('spanish'))
        tokens = [t for t in tokens if t not in stop_words]

        return ' '.join(tokens)

# 3. Búsqueda Dispersa (Sparse Retrieval)


In [5]:
class SparseRetriever:
    """
    Implementa búsqueda dispersa usando BM25.
    """
    def __init__(self, documentos: List[str]):
        self.preprocessor = TextPreprocessor()

        # Preprocesar documentos
        processed_docs = [self.preprocessor.preprocess(doc) for doc in documentos]

        # Tokenizar para BM25
        tokenized_docs = [doc.split() for doc in processed_docs]

        # Inicializar BM25
        self.bm25 = BM25Okapi(tokenized_docs)

        # Guardar documentos originales
        self.documentos = documentos

    def buscar(self, query: str, top_k: int = 3) -> List[Tuple[int, float]]:
        """Realiza búsqueda BM25."""
        # Preprocesar query
        processed_query = self.preprocessor.preprocess(query)
        query_tokens = processed_query.split()

        # Obtener scores BM25
        scores = self.bm25.get_scores(query_tokens)

        # Obtener top_k resultados
        top_indices = np.argsort(scores)[::-1][:top_k]

        return [(idx, scores[idx]) for idx in top_indices]

# 4. Búsqueda Densa (Dense Retrieval)


In [6]:
class DenseRetriever:
    """
    Implementa búsqueda densa usando embeddings.
    """
    def __init__(self, documentos: List[str]):
        # Cargar modelo de embeddings multilingüe
        self.model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

        # Generar y almacenar embeddings
        self.embeddings = self.model.encode(documentos, show_progress_bar=True)

        # Guardar documentos originales
        self.documentos = documentos

    def buscar(self, query: str, top_k: int = 3) -> List[Tuple[int, float]]:
        """Realiza búsqueda por similitud de embeddings."""
        # Generar embedding de la query
        query_embedding = self.model.encode([query])

        # Calcular similitud
        similarities = cosine_similarity(query_embedding, self.embeddings)[0]

        # Obtener top_k resultados
        top_indices = np.argsort(similarities)[::-1][:top_k]

        return [(idx, similarities[idx]) for idx in top_indices]

## 5. Combinación de Técnicas Dispersas y Densas

In [7]:
class HybridRetriever:
    """
    Combina resultados de búsqueda dispersa y densa.
    """
    def __init__(self, documentos: List[str],
                 weight_sparse: float = 0.3,
                 weight_dense: float = 0.7):
        self.sparse_retriever = SparseRetriever(documentos)
        self.dense_retriever = DenseRetriever(documentos)
        self.weight_sparse = weight_sparse
        self.weight_dense = weight_dense
        self.documentos = documentos

    def buscar(self, query: str, top_k: int = 3) -> List[Dict]:
        """
        Realiza búsqueda híbrida y fusiona resultados.
        """
        # Obtener resultados de ambos retrievers
        sparse_results = self.sparse_retriever.buscar(query, top_k=top_k)
        dense_results = self.dense_retriever.buscar(query, top_k=top_k)

        # Combinar scores
        combined_scores = {}
        for idx, score in sparse_results:
            combined_scores[idx] = score * self.weight_sparse

        for idx, score in dense_results:
            if idx in combined_scores:
                combined_scores[idx] += score * self.weight_dense
            else:
                combined_scores[idx] = score * self.weight_dense

        # Ordenar resultados finales
        sorted_results = sorted(combined_scores.items(),
                              key=lambda x: x[1],
                              reverse=True)[:top_k]

        # Preparar resultados
        resultados = []
        for idx, score in sorted_results:
            resultados.append({
                'titulo': df.iloc[idx]['titulo'],
                'descripcion': df.iloc[idx]['descripcion'],
                'genero': df.iloc[idx]['genero'],
                'año': df.iloc[idx]['año'],
                'score': score
            })

        return resultados

# 7. Casos de Uso

Veamos cómo funciona nuestro recuperador de información.

In [8]:
# Inicializamos el retriever disperso
print("Inicializando sistema RAG..")
documentos = df['descripcion'].tolist()
sparse_retriever = SparseRetriever(documentos)
dense_retriever = DenseRetriever(documentos)
hybrid_retriever = HybridRetriever(documentos)


Inicializando sistema RAG..


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

...

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

## 7.1 Búsqueda Dispersa

In [9]:
# basada en keywords
query = "mafia familia italiana nueva york"
print("\nBúsqueda:", query)
sparse_results = sparse_retriever.buscar(query, top_k=3)
print("\nResultados:")
for idx, score in sparse_results:
    print(f"- {df.iloc[idx]['titulo']} ({df.iloc[idx]['año']})")
    print(f"  Score BM25: {score:.3f}")
    print(f"  {df.iloc[idx]['descripcion']}\n")


Búsqueda: mafia familia italiana nueva york

Resultados:
- El Padrino (1972)
  Score BM25: 8.672
  La película sigue a la familia Corleone, una de las familias más poderosas de la mafia italiana en Nueva York.

- El Secreto de Sus Ojos (2009)
  Score BM25: 0.000
  Un oficial judicial se obsesiona con un caso de asesinato sin resolver durante 25 años.

- Cinema Paradiso (1988)
  Score BM25: 0.000
  Un famoso director de cine recuerda su infancia y su primer amor por el cine en un pequeño pueblo italiano.



## 7.2 Búsqueda Densa

In [10]:
# basada en descripciones
query = "películas que exploran la memoria, la nostalgia y el paso del tiempo"
print("\nBúsqueda:", query)
dense_results = dense_retriever.buscar(query, top_k=3)
print("\nResultados:")
for idx, score in dense_results:
    print(f"- {df.iloc[idx]['titulo']} ({df.iloc[idx]['año']})")
    print(f"  Similitud semántica: {score:.3f}")
    print(f"  {df.iloc[idx]['descripcion']}\n")


Búsqueda: películas que exploran la memoria, la nostalgia y el paso del tiempo

Resultados:
- Cinema Paradiso (1988)
  Similitud semántica: 0.363
  Un famoso director de cine recuerda su infancia y su primer amor por el cine en un pequeño pueblo italiano.

- El Padrino (1972)
  Similitud semántica: 0.191
  La película sigue a la familia Corleone, una de las familias más poderosas de la mafia italiana en Nueva York.

- El Laberinto del Fauno (2006)
  Similitud semántica: 0.178
  En la España de 1944, una niña escapa a un mundo mágico y misterioso para huir de su padrastro y la guerra civil.



## 7.3 Búsqueda Híbrida

In [11]:
# basada en combinación de tema y palabras clave
query = "violencia y crimen organizado en latinoamérica"
print("\nBúsqueda:", query)
hybrid_results = hybrid_retriever.buscar(query, top_k=3)
print("\nResultados:")
for res in hybrid_results:
    print(f"- {res['titulo']} ({res['año']})")
    print(f"  Score combinado: {res['score']:.3f}")
    print(f"  {res['descripcion']}\n")



Búsqueda: violencia y crimen organizado en latinoamérica

Resultados:
- Ciudad de Dios (2002)
  Score combinado: 1.539
  La historia del crecimiento del crimen organizado en los suburbios de Río de Janeiro.

- Pulp Fiction (1994)
  Score combinado: 0.284
  Varias historias entrelazadas sobre criminales de Los Ángeles, incluyendo dos hitmen, un boxeador y un gángster.

- El Padrino (1972)
  Score combinado: 0.208
  La película sigue a la familia Corleone, una de las familias más poderosas de la mafia italiana en Nueva York.



# Análisis de los Casos de Uso

1. Búsqueda Dispersa (BM25):
   - Demostró excelente precisión para encontrar coincidencias exactas
   - Encontró rápidamente la película que contenía las palabras clave específicas
   - Ideal para cuando los usuarios saben exactamente qué están buscando

2. Búsqueda Densa (Embeddings):
   - Captó exitosamente conceptos abstractos y temas
   - Encontró películas relevantes incluso sin coincidencias exactas de palabras
   - Perfecta para búsquedas conceptuales y exploratorias

3. Búsqueda Híbrida:
   - Balanceó precisión y comprensión semántica
   - Encontró resultados relevantes combinando palabras clave y contexto
   - Óptima para consultas complejas del mundo real

# Recomendaciones Prácticas:

1. Usa búsqueda dispersa cuando:
   - Los usuarios buscan por términos específicos
   - Necesitas alta precisión en coincidencias exactas
   - Trabajas con recursos computacionales limitados

2. Usa búsqueda densa cuando:
   - Los usuarios hacen consultas conceptuales
   - Necesitas entender el significado más allá de las palabras
   - Tienes recursos para computar embeddings

3. Usa búsqueda híbrida cuando:
   - Tienes casos de uso variados
   - Necesitas robustez ante diferentes tipos de consultas
   - Puedes permitirte el costo computacional adicional

# Extensiones Sugeridas:

1. Mejoras en la búsqueda dispersa:
   - Implementar stemming/lematización
   - Usar n-gramas
   - Probar diferentes variantes de BM25

2. Mejoras en la búsqueda densa:
   - Probar diferentes modelos de embeddings
   - Implementar cross-encoders para re-ranking
   - Usar bases de datos vectoriales (FAISS)

3. Mejoras en la fusión:
   - Implementar métodos más sofisticados de fusión
   - Ajustar pesos dinámicamente
   - Añadir re-ranking

4. Mejoras en la generación:
   - Usar un LLM real
   - Implementar few-shot prompting
   - Añadir control de fuentes